In [12]:


import os
import warnings
import requests
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

load_dotenv()

True

In [13]:
# Set Alpaca API key and secret
# YOUR CODE HERE!
alpaca_api_key = os.getenv("APCA_API_KEY_ID")
alpaca_secret_key = os.getenv("APCA_API_SECRET_KEY")
# Create the Alpaca API object
# YOUR CODE HERE!
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [16]:


now = datetime.now()

# Format the current date in ISO format (YYYY-MM-DD)
today = now.date().isoformat()
# today = pd.Timestamp.now(tz="America/New_York").isoformat()
# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
# YOUR CODE HERE!
current_portfolio_df = api.get_bars(tickers, timeframe, start=today, end=today).df

# Reorganize the DataFrame
# Separate ticker data
agg_df = current_portfolio_df[current_portfolio_df['symbol']=='AGG'].drop(columns='symbol')
spy_df = current_portfolio_df[current_portfolio_df['symbol']=='SPY'].drop(columns='symbol')
# Concatenate the ticker DataFrames
portfolio_concat_df = pd.concat([agg_df, spy_df], axis=1, keys=['AGG', 'SPY'])
# Preview DataFrame
portfolio_concat_df.head()

APIError: subscription does not permit querying recent SIP data